In [1]:
# Basic RBC model with full depreciation (Alternate 1)
#
# Jesus Fernandez-Villaverde
# Haverford, July 3, 2013

import numpy as np
import math
import time
from numba import autojit

# - Start Inner Loop - #
# - bbeta                   float
# - nGridCapital:           int64
# - gridCapitalNextPeriod:  int64
# - mOutput:                float (17820 x 5)
# - nProductivity:          int64
# - vGridCapital:           float (17820, )
# - mValueFunction:         float (17820 x 5)
# - mPolicyFunction:        float (17820 x 5)

@autojit
def innerloop(bbeta, nGridCapital, gridCapitalNextPeriod, mOutput, nProductivity, vGridCapital, expectedValueFunction, mValueFunction, mValueFunctionNew, mPolicyFunction):

    for nCapital in range(nGridCapital):
        valueHighSoFar = -100000.0
        capitalChoice  = vGridCapital[0]
        
        for nCapitalNextPeriod in range(gridCapitalNextPeriod, nGridCapital):
            consumption = mOutput[nCapital,nProductivity] - vGridCapital[nCapitalNextPeriod]
            valueProvisional = (1-bbeta)*np.log(consumption)+bbeta*expectedValueFunction[nCapitalNextPeriod,nProductivity];

            if  valueProvisional > valueHighSoFar:
                valueHighSoFar = valueProvisional
                capitalChoice = vGridCapital[nCapitalNextPeriod]
                gridCapitalNextPeriod = nCapitalNextPeriod
            else:
                break 

        mValueFunctionNew[nCapital,nProductivity] = valueHighSoFar
        mPolicyFunction[nCapital,nProductivity]   = capitalChoice

    return mValueFunctionNew, mPolicyFunction

def main_func():

    #  1. Calibration

    aalpha = 1.0/3.0     # Elasticity of output w.r.t. capital
    bbeta  = 0.95        # Discount factor

    # Productivity values
    vProductivity = np.array([0.9792, 0.9896, 1.0000, 1.0106, 1.0212],float)

    # Transition matrix
    mTransition   = np.array([[0.9727, 0.0273, 0.0000, 0.0000, 0.0000],
                     [0.0041, 0.9806, 0.0153, 0.0000, 0.0000],
                     [0.0000, 0.0082, 0.9837, 0.0082, 0.0000],
                     [0.0000, 0.0000, 0.0153, 0.9806, 0.0041],
                     [0.0000, 0.0000, 0.0000, 0.0273, 0.9727]],float)

    ## 2. Steady State

    capitalSteadyState     = (aalpha*bbeta)**(1/(1-aalpha))
    outputSteadyState      = capitalSteadyState**aalpha
    consumptionSteadyState = outputSteadyState-capitalSteadyState

    print("Output = ", outputSteadyState, " Capital = ", capitalSteadyState, " Consumption = ", consumptionSteadyState)

    # We generate the grid of capital
    vGridCapital           = np.arange(0.5*capitalSteadyState,1.5*capitalSteadyState,0.00001)

    nGridCapital           = len(vGridCapital)
    nGridProductivity      = len(vProductivity)

    ## 3. Required matrices and vectors

    mOutput           = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mValueFunction    = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mValueFunctionNew = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mPolicyFunction   = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    expectedValueFunction = np.zeros((nGridCapital,nGridProductivity),dtype=float)

    # 4. We pre-build output for each point in the grid

    for nProductivity in range(nGridProductivity):
        mOutput[:,nProductivity] = vProductivity[nProductivity]*(vGridCapital**aalpha)

    ## 5. Main iteration

    maxDifference = 10.0
    tolerance = 0.0000001
    iteration = 0

    log = math.log
    zeros = np.zeros
    dot = np.dot

    while(maxDifference > tolerance):

        expectedValueFunction = dot(mValueFunction,mTransition.T)

        for nProductivity in range(nGridProductivity):

            # We start from previous choice (monotonicity of policy function)
            gridCapitalNextPeriod = 0

            # - Start Inner Loop - #

            mValueFunctionNew, mPolicyFunction = innerloop(bbeta, nGridCapital, gridCapitalNextPeriod, mOutput, nProductivity, vGridCapital, expectedValueFunction, mValueFunction, mValueFunctionNew, mPolicyFunction)

            # - End Inner Loop - #

        maxDifference = (abs(mValueFunctionNew-mValueFunction)).max()

        mValueFunction    = mValueFunctionNew
        mValueFunctionNew = zeros((nGridCapital,nGridProductivity),dtype=float)

        iteration += 1
        if(iteration%10 == 0 or iteration == 1):
            print(" Iteration = ", iteration, ", Sup Diff = ", maxDifference)

    return (maxDifference, iteration, mValueFunction, mPolicyFunction)

In [2]:
t1=time.time()
# - Call Main Function - #
maxDiff, iterate, mValueF, mPolicyFunction = main_func()
# - End Timer - #
t2 = time.time()
print(" Iteration = ", iterate, ", Sup Duff = ", maxDiff)
print(" ")
print(" My Check = ", mPolicyFunction[1000-1,3-1])
print(" ")
print("Elapse time = is ", t2-t1)

Output =  0.5627314338711378  Capital =  0.178198287392527  Consumption =  0.3845331464786108
 Iteration =  1 , Sup Diff =  0.0527415934073
 Iteration =  10 , Sup Diff =  0.0313469492659
 Iteration =  20 , Sup Diff =  0.0187034598934
 Iteration =  30 , Sup Diff =  0.011165512034
 Iteration =  40 , Sup Diff =  0.00666854170813
 Iteration =  50 , Sup Diff =  0.00398429274872
 Iteration =  60 , Sup Diff =  0.00238131180393
 Iteration =  70 , Sup Diff =  0.0014236586451
 Iteration =  80 , Sup Diff =  0.000851339774721
 Iteration =  90 , Sup Diff =  0.000509205175229
 Iteration =  100 , Sup Diff =  0.000304623244215
 Iteration =  110 , Sup Diff =  0.000182264856323
 Iteration =  120 , Sup Diff =  0.000109069508726
 Iteration =  130 , Sup Diff =  6.5276437363e-05
 Iteration =  140 , Sup Diff =  3.907108212e-05
 Iteration =  150 , Sup Diff =  2.33880771201e-05
 Iteration =  160 , Sup Diff =  1.40086446374e-05
 Iteration =  170 , Sup Diff =  8.39131720298e-06
 Iteration =  180 , Sup Diff =  5.

In [3]:
# same as above except without print statements

def main_func2():

    #  1. Calibration

    aalpha = 1.0/3.0     # Elasticity of output w.r.t. capital
    bbeta  = 0.95        # Discount factor

    # Productivity values
    vProductivity = np.array([0.9792, 0.9896, 1.0000, 1.0106, 1.0212],float)

    # Transition matrix
    mTransition   = np.array([[0.9727, 0.0273, 0.0000, 0.0000, 0.0000],
                     [0.0041, 0.9806, 0.0153, 0.0000, 0.0000],
                     [0.0000, 0.0082, 0.9837, 0.0082, 0.0000],
                     [0.0000, 0.0000, 0.0153, 0.9806, 0.0041],
                     [0.0000, 0.0000, 0.0000, 0.0273, 0.9727]],float)

    ## 2. Steady State

    capitalSteadyState     = (aalpha*bbeta)**(1/(1-aalpha))
    outputSteadyState      = capitalSteadyState**aalpha
    consumptionSteadyState = outputSteadyState-capitalSteadyState

#     print("Output = ", outputSteadyState, " Capital = ", capitalSteadyState, " Consumption = ", consumptionSteadyState)

    # We generate the grid of capital
    vGridCapital           = np.arange(0.5*capitalSteadyState,1.5*capitalSteadyState,0.00001)

    nGridCapital           = len(vGridCapital)
    nGridProductivity      = len(vProductivity)

    ## 3. Required matrices and vectors

    mOutput           = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mValueFunction    = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mValueFunctionNew = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mPolicyFunction   = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    expectedValueFunction = np.zeros((nGridCapital,nGridProductivity),dtype=float)

    # 4. We pre-build output for each point in the grid

    for nProductivity in range(nGridProductivity):
        mOutput[:,nProductivity] = vProductivity[nProductivity]*(vGridCapital**aalpha)

    ## 5. Main iteration

    maxDifference = 10.0
    tolerance = 0.0000001
    iteration = 0

    log = math.log
    zeros = np.zeros
    dot = np.dot

    while(maxDifference > tolerance):

        expectedValueFunction = dot(mValueFunction,mTransition.T)

        for nProductivity in range(nGridProductivity):

            # We start from previous choice (monotonicity of policy function)
            gridCapitalNextPeriod = 0

            # - Start Inner Loop - #

            mValueFunctionNew, mPolicyFunction = innerloop(bbeta, nGridCapital, gridCapitalNextPeriod, mOutput, nProductivity, vGridCapital, expectedValueFunction, mValueFunction, mValueFunctionNew, mPolicyFunction)

            # - End Inner Loop - #

        maxDifference = (abs(mValueFunctionNew-mValueFunction)).max()

        mValueFunction    = mValueFunctionNew
        mValueFunctionNew = zeros((nGridCapital,nGridProductivity),dtype=float)

        iteration += 1
#         if(iteration%10 == 0 or iteration == 1):
#             print(" Iteration = ", iteration, ", Sup Diff = ", maxDifference)

    return (maxDifference, iteration, mValueFunction, mPolicyFunction)

In [4]:
main_func2()

(9.7160356649084179e-08,
 257,
 array([[-0.9972862 , -0.98551961, -0.97408008, -0.96027188, -0.94819411],
        [-0.99728346, -0.98551687, -0.97407734, -0.96026914, -0.94819137],
        [-0.99728072, -0.98551414, -0.9740746 , -0.96026641, -0.94818864],
        ..., 
        [-0.97049336, -0.95872676, -0.947286  , -0.93347903, -0.92140127],
        [-0.97049244, -0.95872585, -0.94728509, -0.93347812, -0.92140036],
        [-0.97049153, -0.95872494, -0.94728418, -0.93347721, -0.92139945]]),
 array([[ 0.13848914,  0.13996914,  0.14144914,  0.14293914,  0.14443914],
        [ 0.13849914,  0.13996914,  0.14145914,  0.14293914,  0.14443914],
        [ 0.13850914,  0.13997914,  0.14145914,  0.14294914,  0.14444914],
        ..., 
        [ 0.19973914,  0.20185914,  0.20399914,  0.20613914,  0.20830914],
        [ 0.19973914,  0.20185914,  0.20399914,  0.20614914,  0.20830914],
        [ 0.19973914,  0.20185914,  0.20399914,  0.20614914,  0.20830914]]))

In [5]:
%timeit main_func2()

1.39 s ± 140 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit main_func2()

1.33 s ± 4.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit main_func2()

1.32 s ± 7.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit main_func2()

1.34 s ± 23.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
